<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/jyp/spamfiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

In [32]:
df=pd.DataFrame({"tokens":["me free lottery","free get free you","you free scholarship","free to contact me","you won award","you ticket lottery"],
    "spam":["spam","spam","normal","normal","normal","spam"]})
df

,tokens,spam
0,me free lottery,spam
1,free get free you,spam
2,you free scholarship,normal
3,free to contact me,normal
4,you won award,normal
5,you ticket lottery,spam


In [112]:
class spamfiltering:
    def __init__(self,df):
        self.df=df
        
    def make_table(self):
        tokens_ls=[x.split() for x in self.df["tokens"]]
        L=[]
        for i,tokens in enumerate(tokens_ls):
            for word in tokens:
                L.append({"word":word,"tokens_cat":i})
        L=pd.DataFrame(L)
        pivot_table=L.merge(df, how="left",left_on="tokens_cat",right_index=True).pivot_table(index="spam",columns="word",values="tokens",aggfunc="count").fillna(0)
        pivot_table["total"]=pivot_table.sum(axis=1)
        table=pivot_table.T

        return table

    def score(self,k,string):
        log_table=np.log((self.make_table()+k)/(self.make_table().T["total"]+2*k))
        prior_normal=self.make_table().loc["total"]["normal"]/self.make_table().loc["total"].sum()
        prior_spam=self.make_table().loc["total"]["spam"]/self.make_table().loc["total"].sum()

        normal_log_ls=[log_table.loc[word]["normal"] for word in string.split()]
        normal_prob=np.exp(sum(normal_log_ls)+np.log(prior_normal))

        spam_log_ls=[log_table.loc[word]["spam"] for word in string.split()]
        spam_prob=np.exp(sum(spam_log_ls)+np.log(prior_spam))

        return normal_prob/(normal_prob+spam_prob),spam_prob/(normal_prob+spam_prob)

In [113]:
spamfiltering(df).score(0.5,"free lottery")

(0.12500000000000008, 0.8749999999999999)